# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries
import re
import pandas as pd
import numpy as np
import nltk
import pickle
from copy import deepcopy

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer 

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1122)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1122)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1122)>


In [9]:
# load data from database
engine = create_engine('sqlite:///data/messages.db')
df = pd.read_sql_table('messages',engine)
X = df.message
y = df.iloc[:,4:]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    # normalized text and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Extract all the urls from the provided text
    detected_urls = re.findall(url_regex, text)
    # Replace url with a url placeholder string
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    # Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    # List of clean tokens
    return clean_tokens


In [11]:
def create_doc_term_matrix(data_list, vectoriser):
    doc_term_matrix = vectoriser.fit_transform(data_list)
    return pd.DataFrame(doc_term_matrix.toarray(), columns=vectoriser.get_feature_names())

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
#using MultiOutputClassifier to build an ML pipeline
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced', random_state=42))))
])


In [ ]:
pipeline.get_params()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7ff14a5b0700>)),
                ('classifier',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                                                          max_depth=1,
                                                                                                          random_state=42))))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.92      0.65      0.76      6012
               request       0.52      0.77      0.62      1313
                 offer       0.02      0.22      0.03        36
           aid_related       0.71      0.65      0.68      3255
          medical_help       0.32      0.63      0.43       629
      medical_products       0.23      0.65      0.34       406
     search_and_rescue       0.09      0.52      0.15       202
              security       0.04      0.32      0.08       151
              military       0.28      0.69      0.40       244
           child_alone       0.00      0.00      0.00         0
                 water       0.48      0.89      0.62       493
                  food       0.68      0.88      0.77       861
               shelter       0.51      0.80      0.62       715
              clothing       0.14      0.58      0.22       114
                 money       0.17      

/Users/mac/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mac/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mac/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mac/venv/lib/python3.9/site-packages/sklearn/metrics/

In [ ]:
y_test.shape

In [17]:
y_train[y_train.child_alone==1]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix, annot=True)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'vect__smooth_idf': (True, False),
              'vect__use_idf': (True, False),
              'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10],
             'clf__estimator__min_samples_split': [2, 4],}

cv = GridSearchCV(pipeline, parameters)
cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv = GridSearchCV(pipeline, param_grid = parameters,verbose = 2)
np.random.seed(42)
y_pred2 = cv.fit(X_train, y_train)

In [ ]:
prediction2 = y_pred2.predict(X_test)

print(classification_report(y_test, prediction2 , target_names = category_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline2 = Pipeline([
        ('vectorizer', TfidfVectorizer(tokenizer=tokenize)),
        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline2.fit(X_train, y_train)

In [ ]:
y_pred3 = pipeline2.predict(X_test)

print(classification_report(y_test, y_pred3, target_names = category_names))

### 9. Export your model as a pickle file

In [ ]:
# Pickle save model
# It is very hard to find the better performing model especially between the first one and the second one
pickle.dump(y_pred2, open('model.sav', 'wb'))

In [ ]:
### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.